<a href="https://colab.research.google.com/github/jaloaiza/genaiassignments/blob/main/assignments/02/02_TravelAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Gradio Package

In [29]:
# Install Gradio Package
!uv pip install gradio==5.49.1 openai

Using Python 3.12.12 environment at: /usr
Audited 2 packages in 103ms


## Set and Initialize the OpenRouter API Key from Colab Secrets

In [30]:
# Set the OpenRouter API Key
from google.colab import userdata
OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')

# Initialize OpenAI client
import openai
client = openai.OpenAI(
    base_url='https://openrouter.ai/api/v1',
    api_key=OPENROUTER_API_KEY,
)

## Gradio Streaming Chat Interface & Model Selection

In [40]:
import gradio as gr
import json

# By default I prefer Gemini instead of OpenAI
# MUCH faster and cheaper
MODEL = 'google/gemini-2.5-flash-lite' #@param ["openai/gpt-5-mini", "google/gemini-2.5-flash-lite"]

SYSTEM_PROMPT = {
    "role": "system",
    "content": (
        "You are a helpful, concise Washington State Trip Planner. "
        "You specialize in planning trips within Washington State, "
        "answering travel questions, and suggesting destinations, itineraries, and budgets. "
        "When producing an itinerary, be specific with times, transit, and costs."
    )
}

FINALIZE_TRIGGERS = {"finalize", "generate itinerary", "itinerary", "/itinerary", "done"}


TRIP_SCHEMA = {
    "name": "trip",
    "schema": {
        "type": "object",
        "additionalProperties": False,
        "properties": {
            "destination": {"type": "string"},
            "duration_days": {"type": "integer"},
            "budget_level": {"type": "string", "enum": ["low", "mid", "high"]},
            "activities": {"type": "array", "items": {"type": "string"}},
            "daily_schedule": {
                "type": "array",
                "items": {
                    "type": "object",
                    "additionalProperties": False,
                    "properties": {
                        "day": {"type": "integer"},
                        "segments": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "additionalProperties": False,
                                "properties": {
                                    "time": {"type": "string"},
                                    "activity": {"type": "string"},
                                    "location": {"type": "string"},
                                    "transit": {"type": "string"},
                                    "cost_notes": {"type": "string"}
                                },
                                "required": ["time", "activity", "location"]
                            }
                        }
                    },
                    "required": ["day", "segments"]
                }
            }
        },
        "required": [
            "destination",
            "duration_days",
            "budget_level",
            "activities",
            "daily_schedule"
        ]
    }
}

def format_trip(trip: dict) -> str:
    lines = []
    lines.append(f"##  {trip['destination']} — {trip['duration_days']} days ({trip['budget_level']} budget)\n")

    if trip.get("activities"):
        lines.append("**Highlights:** " + ", ".join(trip["activities"]) + "\n")

    for day in trip["daily_schedule"]:
        lines.append(f"\n### Day {day['day']}")
        for seg in day["segments"]:
            lines.append(
                f"- **{seg.get('time','')}** — **{seg['activity']}**  \n"
                f"   {seg['location']}"
            )
            if seg.get("transit"):
                lines.append(f"   {seg['transit']}")
            if seg.get("cost_notes"):
                lines.append(f"   {seg['cost_notes']}")

    return "\n".join(lines)


def chat_with_streaming(message, history):
    message_clean = (message or "").strip().lower()

    if message_clean in FINALIZE_TRIGGERS:
        try:
            messages = (
                [SYSTEM_PROMPT]
                + history
                + [{
                    "role": "user",
                    "content": (
                        "Using the full conversation so far, generate the final trip itinerary now. "
                        "Return ONLY valid JSON matching the provided schema."
                    )
                }]
            )

            resp = client.chat.completions.create(
                model=MODEL,
                messages=messages,
                response_format={"type": "json_schema", "json_schema": TRIP_SCHEMA},
            )

            trip = json.loads(resp.choices[0].message.content)

            md = format_trip(trip)

            yield (
                "Itinerary generated! See the panels ABOVE.",
                md,       # goes to gr.Markdown
                trip      # goes to gr.JSON
            )
            return

        except Exception as e:
          print("Finalize exception:", repr(e))
          yield f"Finalize failed: `{type(e).__name__}: {e}`"
          return


    # normal streaming
    messages = [SYSTEM_PROMPT] + history + [{"role": "user", "content": message}]

    stream = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        stream=True,
    )

    response_text = ""
    for chunk in stream:
        delta = chunk.choices[0].delta
        if getattr(delta, "content", None):
            response_text += delta.content
            # While streaming, leave itinerary panels unchanged
            yield (response_text, gr.update(), gr.update())

with gr.Blocks() as demo:
    gr.Markdown("# Washington State Travel Agent")

    itinerary_md = gr.Markdown(label="Itinerary (Markdown)")
    itinerary_json = gr.JSON(label="Itinerary (Structured JSON)")

    chat = gr.ChatInterface(
        fn=chat_with_streaming,
        type="messages",
        title="Chat",
        description="Chat normally. Type **finalize** to generate the itinerary.",
        additional_outputs=[itinerary_md, itinerary_json]
    )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://15f46df6cebb848618.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
